# ZOO dataset

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as tf_layers

import dgates

## Loading data

In [3]:
data = pd.read_csv('data/zoo.csv')

In [4]:
data

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,wallaby,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,wasp,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,wolf,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,worm,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


In [5]:
legs = data['legs']
other_features = data.drop(columns=['animal_name', 'legs', 'class_type']).to_numpy()
legs_onehot = OneHotEncoder().fit_transform(legs.to_numpy().reshape(-1, 1)).todense()
features = np.hstack([other_features, legs_onehot])
features = np.array(features.astype(bool))

In [6]:
y = data['class_type'].to_numpy()

is_mammal = y == 1
is_bird = y == 2
is_reptile = y == 3
is_fish = y == 4
is_amphibian = y == 5
is_bug = y == 6

In [7]:
features

array([[ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False,  True,  True, ..., False, False, False]])

In [8]:
outputs = is_mammal.reshape(-1, 1)

## Diff Gates

In [10]:
network = dgates.Network(21, [40, 50, 50, 40, 1], connections_rate=10)

In [11]:
loss, accuracy = network.evaluate_real(features, outputs)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Loss: 0.243 Accuracy: 0.594


In [12]:
loss, accuracy = network.evaluate_boolean(features, outputs)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Loss: 0.436 Accuracy: 0.564


In [13]:
network.fit(features, is_mammal, epochs=100, learning_rate=0.1)

Epoch 100: 100%|██████████| 101/101 [00:01<00:00, 66.08it/s]


In [14]:
loss, accuracy = network.evaluate_real(features, outputs)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Loss: 0.002 Accuracy: 1.000


In [15]:
loss, accuracy = network.evaluate_boolean(features, outputs)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Loss: 0.000 Accuracy: 1.000


## Gates

In [ ]:
from gates import gates
from gates import optimizers as gates_optimizers
from gates.optimizers import genetic as gates_genetic

In [24]:
network = gates.Network(21, [40, 50, 50, 40, 1])

In [27]:
optimizer = gates_optimizers.GeneticOptimizer(
    fitness=gates_genetic.LowestMseFitness(),
    crossing=gates_genetic.MultiPointCrossing(3),
    mutation=gates_genetic.NRandomChangesMutation(4),
    selection=gates_genetic.RouletteWheelSelection(),
    cross_prob=0.7,
    mutate_prob=0.2,
    n_chromosomes=100,
    n_generations=200,
    n_elite=4,
    patience=50,
)

optimizer.optimize(network, features, outputs)

Optimizing 905 parameters


Evolution:  14%|█▍        | 28/200 [00:21<02:09,  1.33it/s, fitness: inf, mse: 0.000, accuracy: 1.000]              


In [28]:
loss, accuracy = network.evaluate(features, outputs)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Loss: 0.000 Accuracy: 1.000


## Neural Network

In [32]:
network = keras.models.Sequential([
    tf_layers.InputLayer((21,)),
    tf_layers.Dense(40, activation='relu'),
    tf_layers.Dense(50, activation='relu'),
    tf_layers.Dense(50, activation='relu'),
    tf_layers.Dense(40, activation='relu'),
    tf_layers.Dense(1 , activation=None)
])

In [23]:
network.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 40)                880       
                                                                 
 dense_16 (Dense)            (None, 50)                2050      
                                                                 
 dense_17 (Dense)            (None, 50)                2550      
                                                                 
 dense_18 (Dense)            (None, 40)                2040      
                                                                 
 dense_19 (Dense)            (None, 1)                 41        
                                                                 
Total params: 7,561
Trainable params: 7,561
Non-trainable params: 0
_________________________________________________________________


In [33]:
network.compile(optimizer='adam', loss='mse', metrics=['binary_accuracy'])

In [34]:
network.fit(
    features,
    outputs,
    validation_data=(features, outputs),
    epochs=2,
    batch_size=1,
)

Epoch 1/2
101/101 [==============================] - 0s 2ms/step - loss: 0.0837 - binary_accuracy: 0.9208 - val_loss: 0.0113 - val_binary_accuracy: 1.0000
Epoch 2/2
101/101 [==============================] - 0s 1ms/step - loss: 0.0072 - binary_accuracy: 1.0000 - val_loss: 0.0022 - val_binary_accuracy: 1.0000


In [38]:
loss, accuracy = network.evaluate(features, outputs, verbose=0)
print(f'Loss: {loss:.3f} Accuracy: {accuracy:.3f}')

Loss: 0.002 Accuracy: 1.000
